# Tile Tensor Tutorial

<br>

In [ ]:
import unittest
import numpy as np
import sys
import pyhelayers
import utils 

utils.verify_memory()

<br>

#### Initialize the HeContext

In [ ]:
requirement = pyhelayers.HeConfigRequirement(
    num_slots = 4,
    multiplication_depth = 10,
    fractional_part_precision = 50,
    integer_part_precision = 10)
requirement.security_level = 0 # insecure

he_context = pyhelayers.DefaultContext()
he_context.init(requirement)

<br>

#### Encrypt two 2x3 tensors

In [ ]:
shape = pyhelayers.TTShape()
shape.add_dim(pyhelayers.TTDim(2, 2))
shape.add_dim(pyhelayers.TTDim(3, 2))

In [ ]:
tt_encoder = pyhelayers.TTEncoder(he_context)

vals1 = range(6)
vals1 = np.reshape(vals1, (2, 3))
ctt1 = tt_encoder.encode_encrypt(shape, vals1)

In [ ]:
vals2 = [i + 6 for i in range(6)]
vals2 = np.reshape(vals2, (2, 3))
ctt2 = tt_encoder.encode_encrypt(shape, vals2)

<br>

#### Perform HE operations

In [ ]:
ctt1.add(ctt2)

In [ ]:
add_res = tt_encoder.decrypt_decode_double(ctt1)
print("We added encryption of:")
print(vals1)
print("With encryption of:")
print(vals2)
print("Resulting with encryption of:")
print(add_res)

In [ ]:
ctt1.multiply(ctt2)

In [ ]:
mul_res = tt_encoder.decrypt_decode_double(ctt1)
print("We multiplied encryption of:")
print(add_res)
print("With encryption of:")
print(vals2)
print("Resulting with encryption of:")
print(mul_res)

<br>

#### Generate a random tensor
All values are in the range [500, 1000).

In [ ]:
shape = pyhelayers.TTShape()
shape.add_dim(pyhelayers.TTDim(2, 2))
shape.add_dim(pyhelayers.TTDim(3, 2))
lower_bound = 500
upper_bound = 1000
vals3 = np.random.default_rng().uniform(lower_bound,
 upper_bound,
 shape.get_original_sizes())
print(vals3)

<br>

#### Encrypt the random tensor

In [ ]:
tt_encoder = pyhelayers.TTEncoder(he_context)
tt = tt_encoder.encode_encrypt(shape, vals3)

<br>

#### Compute element-wise inverse of "tt" CTileTensor
Note that the inverse function expects to receive a lower and an upper bound on all of the values of the given CTileTensor. See FunctionEvaluator::inverse() for more details.

In [ ]:
fe = pyhelayers.TTFunctionEvaluator(he_context)
tt = fe.inverse(tt, lower_bound, upper_bound)

<br>

#### Decrypt and decode results

In [ ]:
inverse_res = tt_encoder.decrypt_decode_double(tt)
print("We computed elementwise inverse of:")
print(vals3)
print("Resulting with encryption of:")
print(inverse_res)

<br>

#### Duplicate over dimension
For some computations it is useful to have the tensor data replicated several times inside the tile slots. Meaning that for some dimension, the original size is 1, but each element of the original tensor is replicated along this dimension. When performing operations with another Tile Tensor with compatible shape (namely, The tensors A[n1,...,nk] and B[m1,...,mk] have compatible shapes if mi =ni or either ni =1 or mi =1, for i≤k), we automatically expend the replicated dimensions to match larger size.

In [ ]:

# This defines the basic tile shape. 
# It means we'll use each ciphertext as a tile of size 2x2 (each ciphertext has 4 slots).
baseShape = pyhelayers.TTShape([2,2])

# For the first tensor, we defined both dimensions to be duplicated.
shape1=baseShape.get_with_duplicated_dims([0,1])

# For the second tensor, we'll just use the base shape:
shape2=baseShape

Now we'll define the tensors we want to use: 

In [ ]:
vals1 = np.random.default_rng().uniform(1, 2, [1,1])
vals2 = np.random.default_rng().uniform(1, 2, [165,94])

And pack them into the tile tensors with their respective shapes:

In [ ]:
tt_encoder = pyhelayers.TTEncoder(he_context)

ctt1 = tt_encoder.encode_encrypt(shape1, vals1)
ctt2 = tt_encoder.encode_encrypt(shape2, vals2)

# We can print the shapes of the two tile tensors:
print('Encrypted tile tensor 1 shape:',ctt1.get_shape())
print('Encrypted tile tensor 1 size in bytes:',len(ctt1.save_to_buffer()))
print('Encrypted tile tensor 2 shape:',ctt2.get_shape())
print('Encrypted tile tensor 2 size in bytes:',len(ctt2.save_to_buffer()))

In the prinout of the shapes, the numbers to the left of the / indicate the tensor size, or * if it is duplicated.
So one tensor is fully duplicated (two *), and the other has size [2,3].
The numbers to the right of the / indicate the size of the tiles used to store the tensors. 

Since the shapes are compatible, we can now multiply.

In [ ]:
ctt1.multiply(ctt2)

Notice that the shape of the first tensor changed.

In [ ]:
print('Encrypted tile tensor 1 new shape:',ctt1.get_shape())

And now we can decrypt the result:

In [ ]:
res = tt_encoder.decrypt_decode_double(ctt1)
print(res)

<br>

#### Matrix multiplication
Here we demonstrate how to perform a matrix multiplication of two matrices of size 3X5 and 5X2 efficiently. We accomplish this by using the 'Duplicate over dimension' feature (see explanation above), which allows us to duplicate values along a specific dimension without setting them manually.

In [ ]:
requirement = pyhelayers.HeConfigRequirement(
    num_slots = 32,
    multiplication_depth = 10,
    fractional_part_precision = 50,
    integer_part_precision = 10)
requirement.security_level = 0 # insecure

he_context = pyhelayers.DefaultContext()
he_context.init(requirement)

In [ ]:
# Common base shape for both matrices
baseShape=pyhelayers.TTShape([4,2,4]) # Tile sizes only
shape1=baseShape.get_with_duplicated_dim(2)
shape2=baseShape.get_with_duplicated_dim(0)

In [ ]:
vals1 = np.random.default_rng().uniform(1, 2, [3,5,1])
vals2 = np.random.default_rng().uniform(1, 2, [1,5,2])

In [ ]:
tt_encoder = pyhelayers.TTEncoder(he_context)
ctt1 = tt_encoder.encode_encrypt(shape1, vals1)
ctt2 = tt_encoder.encode_encrypt(shape2, vals2)

In [ ]:
ctt1.multiply_and_sum(ctt2, 1)

In [ ]:
res = tt_encoder.decrypt_decode_double(ctt1)
print(f"Mean error: {np.linalg.norm(np.dot(vals1[:, :, 0],vals2[0, :, :])-res[:, 0, :]):.16f}")